In [0]:
#Imports
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f

#Dataset Schema
schema = StructType([
  StructField("order_id", StringType()),
  StructField("timestamp", StringType()),
  StructField("customer_id", StringType()),
  StructField("product_id", StringType()),
  StructField("category", StringType()),
  StructField("price", DoubleType()),
  StructField("quantity", IntegerType()),
  StructField("total_amount", DoubleType()),
  StructField("city", StringType()),
  StructField("state", StringType()),
  StructField("country", StringType()),
  StructField("latitude", StringType()),
  StructField("longitude", StringType()),
  StructField("delivery_status", StringType())])

  # Azure Event Hub Configuration
event_hub_namespace='mn-ecommerce-namespace'
event_hub_name='ecommerce-orders'
shared_access_key='xxxxxxxx'
event_hub_conn_str='Endpoint=sb://mn-ecommerce-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey={shared_access_key}'

eh_conf = {
    'kafka.bootstrap.servers':f'{event_hub_namespace}.servicebus.windows.net:9093',
      'subscribe': event_hub_name,
          'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
          'kafka.sasl.mechanism': 'PLAIN',
          'kafka.security.protocol':  'SASL_SSL',
          'startingOffsets': 'latest',
          'failOnDataLoss': 'false'
}

#Stream from Event hub
df_raw = (
    spark.readStream
    .format("kafka")
    .options(**eh_conf)
    .load()
)

#Stream read is in bytes so we need to convert it to string
#Parse json from kafka stream
df_orders = (
    df_raw.selectExpr('Cast(value AS STRING) as json')
    .select(from_json("json", schema).alias("data"))
    .select("data.*")
)

#access the bronze table 
#we can use either key or string based access to accees the azure storage account (Using key based access)
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
  "<storage_account_access_key>"
)

#specifying path to write data
bronze_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/bronze"

(
    df_orders.writeStream
    .format('delta')
    .outputMode("append")
    .option("checkpointLocation",bronze_path+"/_checkpoint")
    .start(bronze_path)
)

In [0]:
df_bronze = spark.read.format('delta').load(bronze_path)
display(df_bronze.limit(10))

order_id,timestamp,customer_id,product_id,category,price,quantity,total_amount,city,state,country,latitude,longitude,delivery_status
ed06cd32-d44f-41d5-bb01-dbe56bd51ba8,2025-11-24T17:26:02.534984,b688c11a-a6c6-45a6-9b87-b7efaf3f6bf1,a24807e9-5297-4a5c-9316-25d8242a575b,Books,1333.24,4,5332.96,Chicago,IL,USA,41.8781,-87.6298,Processing
9fc52ade-fd8b-4da1-8f1f-79cc8a664762,2025-11-24T17:26:04.536267,2f4bb79c-40f4-4296-8e96-853b572bee95,6c146329-c965-4c86-97ff-a9d0b6b3696d,Toys,279.33,3,837.99,Houston,TX,USA,29.7604,-95.3698,Shipped
6cc23525-d1c9-4fd1-89f5-841c8ef6d7f5,2025-11-24T17:26:06.537315,173a4cc9-39f0-43eb-9c9f-672ea29b7b9f,f81cbed6-cabb-498f-8547-fced02b66b07,Home Decor,409.58,5,2047.9,Los Angeles,CA,USA,34.0522,-118.2437,Delivered
772c7ee7-1735-44a2-8839-4ae4d42fbc5c,2025-11-24T17:26:08.538099,d7be1c48-4135-420f-b9c3-3f051acb5589,fc94424c-cd65-4c92-98ef-88815c8591bb,Clothing,828.48,4,3313.92,Chicago,IL,USA,41.8781,-87.6298,Cancelled
d3a3d1a9-4141-4795-8020-ee02540dbf8b,2025-11-24T17:26:10.538869,5698834e-4587-412f-8e5c-03712d251d52,a0a57b0a-1f41-40da-9c79-60a10190399b,Books,73.71,4,294.84,New York,NY,USA,40.7128,-74.006,Shipped
e2aaad4d-4208-4ddf-9b0b-e74e0c60765b,2025-11-24T17:26:12.539707,4661861c-c57f-474f-b814-e0abd626477b,0a0f6ad6-b880-41d5-a777-cc68cbfae8e9,Clothing,1918.58,1,1918.58,New York,NY,USA,40.7128,-74.006,Cancelled
019fc176-be12-4a7a-b912-10be1e003554,2025-11-24T17:26:14.540596,3f345212-63f2-4f22-937c-8a76b68e9328,685cdd65-fb36-4777-aa39-2902df9b6c6b,Electronics,1442.26,4,5769.04,Phoenix,AZ,USA,33.4484,-112.074,Delivered
ce26009a-e4c1-4200-8f5d-802a1fe2c591,2025-11-24T17:26:16.541501,50eab753-7e91-473d-8f01-9510d2a3b6e6,9eb30498-ca64-460b-ba8d-1c9ecf08b8d3,Books,612.61,4,2450.44,Los Angeles,CA,USA,34.0522,-118.2437,Shipped
4cddefdf-9464-43d1-ba07-893a5d54b1b9,2025-11-24T17:26:18.542203,380d7daf-07ca-430c-833b-d533d750a083,9bc4609a-6694-4892-8142-16f70bda0319,Books,128.97,2,257.94,Chicago,IL,USA,41.8781,-87.6298,Cancelled
d04200ba-088e-4601-9b48-f801a7f78b8d,2025-11-24T17:26:20.543066,443add87-da21-4e17-b0d5-9854071699cd,552c2d4f-4f3b-43bf-b6bf-b731ee0cfeb2,Home Decor,1333.72,4,5334.88,New York,NY,USA,40.7128,-74.006,Shipped
